# Basic Mapping
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/htcondor/htmap/master?urlpath=lab/tree/tutorials/basic-mapping.ipynb)

## Tags

In the previous tutorial, we used HTMap like this:

In [ ]:
import htmap

def double(x):
    return 2 * x

In [ ]:
mapped = htmap.map(double, range(10))
print(mapped)
doubled = list(mapped)
print(doubled)

In particular, we used the [htmap.map](https://htmap.readthedocs.io/en/stable/api.html#htmap.map) function to create our map.
This function creates an object that behaves a lot like the iterator returned by the built-in [map](https://docs.python.org/3/library/functions.html#map) function.
To get our output, we iterated over it using `list`.

You may have noticed that the map has a **tag** associated with it.
HTMap generated this tag for us because we didn't provide one, and because we didn't provide one, marked the map as **transient**, as opposed to **persistent**.
Transient maps are for quick tests where we don't care too much about organization.
Persistent maps are for longer-running maps where we want to keep our work organized by giving things real names.
If you don't plan on using your map for more than one session, you can probably get away with a transient map.
If you're going to step away from the computer and come back, we recommend giving it a real tag.

The map we created above is transient:

In [ ]:
print(mapped.is_transient)

To create a persistent map, we need to give our map our map a tag:

In [ ]:
another_map = htmap.map(double, range(10), tag = 'dbl')
print(another_map)
print(another_map.is_transient)

We can also "retag" a map to give it a new tag.
If you tag a transient map, it becomes persistent.

In [ ]:
mapped.retag('a-new-tag')
print(mapped)
print(mapped.is_transient)

## Working with Maps

The object that was returned by [htmap.map](https://htmap.readthedocs.io/en/stable/api.html#htmap.map) is a [htmap.Map](https://htmap.readthedocs.io/en/stable/api.html#htmap.Map).
It gives us a window into the map as it is running, and lets us use the output once the map is finished.

For example, we can print the status of the map:

In [ ]:
stringified = htmap.map(str, range(10), tag = 'str')
print(stringified.status())

We can wait for the map to finish:

In [ ]:
stringified.wait(show_progress_bar = True)

There are many ways to iterate over maps:

In [ ]:
print(list(stringified))

for d in stringified:
    print(d)

If we ever lose our reference to it, we can grab a new reference to it using [htmap.load](https://htmap.readthedocs.io/en/stable/api.html#htmap.load), giving it the tag of the map we want:

In [ ]:
new_ref = htmap.load('str')

print(new_ref)
print(new_ref == stringified)
print(new_ref is stringified)  # maps are singletons

Maps can be recovered from an entirely different Python interpreter session as well.
Suppose you close Python and go on vacation.
You come back and you want to look at your map again, but you've forgotten what you called it.
Just ask HTMap for a list of your tags:

In [ ]:
print(htmap.get_tags())

Ok, well, technically it was a tuple, but we'll have to live with it.

HTMap can also print a pretty table showing the status of your maps:

In [ ]:
htmap.map(str, range(5))  # new transient map
print(htmap.status())

Note that transient maps have a `*` in front of their tags.

The status message tells us about how many components of our map are in each of the five most common component states:

* Idle - component is waiting to run
* Running - component is currently executing remotely
* Completed - component is finished executing and output is available
* Held - HTCondor has noticed a problem with the component and is not letting it run
* Errored - there was an error in your code, and HTMap has brought back error information

The status of each component of your map is available using the map attribute `component_statuses`:

In [ ]:
print(new_ref.component_statuses)

We'll discuss what to do about held and errored components and how to interact with component statuses in the [Error Handling](error-handling.ipynb) tutorial.

Tags are *unique*: if we try to create another map with a tag we've already used, it will fail:

In [ ]:
new_map = htmap.map(double, range(10), tag = 'dbl')

As the error message indicates, if we want to re-use the tag `dbl`, we need to remove the old map first:

In [ ]:
old_map = htmap.load('dbl')
old_map.remove()

[htmap.Map.remove](https://htmap.readthedocs.io/en/stable/api.html#htmap.Map.remove) deletes all traces of the map.
**It can never be recovered**.
Be careful when using it!

The module-level shortcut [htmap.remove](https://htmap.readthedocs.io/en/stable/api.html#htmap.remove) lets you skip the intermediate step of getting the actual Map, if you don't already have it.

Now we can re-use the map ID:

In [ ]:
new_map = htmap.map(double, range(10), tag = 'dbl')
new_map.wait(show_progress_bar = True)
print(list(new_map))

## Map Builders

So far we've been avoiding any functions that needed to be mapped over keyword arguments, or that had more than one positional argument.
`htmap.map` is not really the ideal tool for working with functions that have more than one argument, and it does not support varying more than one argument at all.

A much more ergonomic way to build up a complex map is a **map builder**.
A map builder lets you build a map via individual function calls.
Call [htmap.build_map](https://htmap.readthedocs.io/en/stable/api.html#htmap.build_map) as a context manager to get the builder, then call the builder as if it were the mapped function itself:

In [ ]:
def power(base, exponent):
    return base ** exponent

with htmap.build_map(power) as pow_builder:
    for base in range(1, 5):  # bases are 1, 2, 3, 4
        for exponent in range(1, 4):  # exponents are 1, 2, 3
            pow_builder(base, exponent)

powered = pow_builder.map
print(list(powered))  # 1^1, 1^2, 1^3, 2^1, 2^2, 2^3, 3^1 ...

The map builder catches the function calls and turns them into a map.
The map is created when the `with` block ends, at which point you can grab the actual [htmap.Map](https://htmap.readthedocs.io/en/stable/api.html#htmap.Map) from the builder's `map` attribute.

---
In the [next tutorial](working-with-files.ipynb), we'll see how to tell HTMap to bring a local file along to the execute node.